In [4]:
!pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-win_amd64.whl size=1291613 sha256=dad2ecd7f3c3401f529f527790d7a84775784752c843f38459e2db0ab47a2569
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\75\fa\bc\739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

In [3]:
#specify correct data types for each column
dtype_mapping={"user_id":int ,"item_id":int ,"rating":float, "timestamp":str}

ratings_data = pd.read_csv("rating.csv", names=["user_id", "item_id", "rating", "timestamp"])

# Drop timestamp as it's not needed
ratings_data = ratings_data.drop("timestamp", axis=1)

# Check data
print(ratings_data.head())
#check the datatype
print(ratings_data.dtypes)

C:\Users\DELL\AppData\Local\Temp\ipykernel_6196\739447550.py:4: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_data = pd.read_csv("rating.csv", names=["user_id", "item_id", "rating", "timestamp"])


  user_id  item_id  rating
0  userId  movieId  rating
1       1        2     3.5
2       1       29     3.5
3       1       32     3.5
4       1       47     3.5
user_id    object
item_id    object
rating     object
dtype: object


In [4]:
ratings_data["user_id"] = pd.to_numeric(ratings_data["user_id"], errors="coerce")
ratings_data["item_id"] = pd.to_numeric(ratings_data["item_id"], errors="coerce")
ratings_data["rating"] = pd.to_numeric(ratings_data["rating"], errors="coerce")

# Drop any rows where conversion failed (NaN values)
ratings_data = ratings_data.dropna()

# Convert to integers
ratings_data["user_id"] = ratings_data["user_id"].astype(np.int64)
ratings_data["item_id"] = ratings_data["item_id"].astype(np.int64)
ratings_data["rating"] = ratings_data["rating"].astype(float)

# Check again
print(ratings_data.dtypes)

user_id      int64
item_id      int64
rating     float64
dtype: object


In [5]:
# Define Reader object with rating scale
reader = Reader(rating_scale=(1, 5))

# Load data into Surprise dataset
data = Dataset.load_from_df(ratings_data[['user_id', 'item_id', 'rating']], reader)

In [ ]:
# Use SVD (Singular Value Decomposition) algorithm
svd = SVD()

# Perform cross-validation
cross_validate(svd, data, cv=5, verbose=True)

In [ ]:
# Train the model on full dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

# Predict a rating for a specific user and item
user_id = 196
item_id = 302
pred = svd.predict(uid=user_id, iid=item_id)
print(pred.est)  # Estimated rating

In [ ]:
# Get all unique items
all_items = ratings_data['item_id'].unique()

# Get already rated movies
rated_items = ratings_data[ratings_data['user_id'] == user_id]['item_id'].values

# Predict ratings for unrated items
predictions = [(iid, svd.predict(uid=user_id, iid=iid).est) for iid in all_items if iid not in rated_items]

# Sort predictions by rating
top_recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

print("Top 5 Recommended Movies for User", user_id, ":", top_recommendations)
